Welcome everyone this afternoon session. This worksheep will guide you to everything that you need including instructions, the questions and definitely the bonus questions for your extra scores. Before we start with anything, please fill in your information here so we can give you nice some nice scores later. 

- Member1:
- Member2: 
- Member3: 
- Member4: 
- Member5: 


Please try to elaborate this exercise with the lectures from the first week. The goal of this lab is for you to enjoy mass spectrometry data much more. MS is absolutely an intrigueing technology. In other words, it is not that simple to understand it with in a few minutes but we are here to explore them all together. Nevertheless, we try to reduce several steps into just a simple exercise. 

## Intended learning outcomes (ILOs)

On completion of the lab, the student should be able to:

1. Demonstrate understandings and insights in mass spectrometry
2. Retrieve open access mass spectrometry data
3. Access organism proteome from database 
4. Identify the relevant issues of complexity in high-throughput data structure from mass spectrometry
5. Demonstrate capacity to store and handle high throughput data with efficiency 

## Let's start! 

We have created a simple exercise for you on <https://github.com/thanadol-git/bc_25>. Please try to follow it along with answering the questions here. The teachers will try to help you in the class so don't worry if it looks a bit difficult. We are always here to help. 

You may learn a bit about mass spectrometry from Fredrik. Now, we are touching a real dataset. First look at the SDRF file. Here is the [link](https://www.cell.com/cancer-cell/fulltext/S1535-6108(22)00274-4?_returnURL=https%3A%2F%2Flinkinghub.elsevier.com%2Fretrieve%2Fpii%2FS1535610822002744%3Fshowall%3Dtrue) to the main article.

## Q1. 
**What do you see in the SDRF file? How many samples are there?** 

Ans.

## Q2.
**What is PRIDE accession ID for this dataset? Summarise the experiment in a short paragraph (<100 words).**

Ans. 

## Q3. 
**What samples were used and located in the organism/s in this experiment?**

Ans.

## Q4.
**What is the model of the instrument? What is its set-up? Is it labelled or labelled-free quantification?**

Ans. 


In [ ]:
# libraries 
import numpy as np
import pandas as pd
# matplotlib
import matplotlib.pyplot as plt
# seaborn
import seaborn as sns
# sklearn

In [ ]:
# Download the results 

## SDRF
sdrf = pd.read_csv('cell_metadata_scbc25.sdrf.tsv', sep='\t')
## DIANN report
diann_report = pd.read_csv('diannsummary/diann_report.unique_genes_matrix.tsv', sep='\t')


## Q5. 
**What do you see in the report file? What are the columns and the rows** 



In [ ]:
# Dynamic range

# pivot longer of all column  except Genes as `samples`
diann_report_long = diann_report.melt(id_vars=['Genes'], var_name='Samples', value_name='Intensity')

diann_report_long.head()

In [ ]:
n_samples = diann_report_long['Samples'].nunique()
print(f'Number of samples: {n_samples}')

In [ ]:
# Count missing values for each genes
missing_values = diann_report_long.groupby('Genes')['Intensity'].apply(lambda x: x.isna().sum()).reset_index()
# rename Intensity to Missing
missing_values.rename(columns={'Intensity': 'Missing'}, inplace=True)
# remove Missing is zero
# missing_values = missing_values[missing_values['Missing'] > 0]
# sort by Missing   
missing_values.sort_values(by='Missing', ascending=False, inplace=True)
# reset index
missing_values.reset_index(drop=True, inplace=True)

# Calculate percentage of missing values
missing_values['Percentage'] = (missing_values['Missing'] / n_samples) * 100

missing_values.head()


## Q6. 
**How many genes that are always being detected in this experiment?** 


In [ ]:
# Filter missing genes
# Use 50% rule
selected_genes = missing_values[missing_values['Percentage'] <= 50]['Genes'].tolist()
print(selected_genes)

# Filter diann_report_long for selected genes
diann_report_long_filtered = diann_report_long[diann_report_long['Genes'].isin(selected_genes)]

# impute missing values with 0
diann_report_long_filtered['Intensity'].fillna(0, inplace=True)
diann_report_long_filtered.head()

In [ ]:
# Join with sdrf
dr_plot_join = diann_report_long_filtered.merge(sdrf, left_on='Samples', right_on='comment[data file]', how='left')

# filter only non zero intensity
dr_plot = dr_plot_join[dr_plot_join['Intensity'] > 0]

In [ ]:
# Plot the dynamic range
plt.figure(figsize=(12, 6))
sns.boxplot(data=dr_plot, x='factor value[cell line]', y='Intensity', hue = 'characteristics[organism part]',
            palette='Set2', showfliers=False)
plt.xticks(rotation=90)
plt.title('Dynamic Range of Intensities Across Samples')
plt.xlabel('Samples')
plt.ylabel('Intensity')
plt.tight_layout()
plt.yscale('log')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()


In [ ]:
# Find average intensity per genes
avg_intensity = dr_plot.groupby('Genes')['Intensity'].mean().reset_index()
# Sort by average intensity
avg_intensity.sort_values(by='Genes', ascending=True, inplace=True)
# Reset index
avg_intensity.reset_index(drop=True, inplace=True)  
avg_intensity.head(10)

In [ ]:
# Find average expression per tissue
avg_intensity_tissue = dr_plot.groupby(['Genes', 'characteristics[organism part]'])['Intensity'].mean().reset_index()
#reanme column Intensity to Average_Intensity
avg_intensity_tissue.rename(columns={'Intensity': 'Average_Tissue_Intensity'}, inplace=True)

avg_intensity_tissue.head()

Now we want to see the enrich genes for each tissue. We just basically calculate the ratio betweeb average gene expression and the average expression of gene for all tissues.

In [ ]:
# Calculate tissue ratio of average in each gene

# join avg_intensity with avg_intensity_tissue
avg_intensity_joined = avg_intensity.merge(avg_intensity_tissue, on='Genes', how='left')

# Calculate ratio
avg_intensity_joined['Tissue_Ratio'] = avg_intensity_joined['Average_Tissue_Intensity'] / avg_intensity_joined['Intensity']

avg_intensity_joined


In [ ]:
# Look at your tissue 
tissue = 'Skin'
# Filter by tissue
top_marker_skin = avg_intensity_joined[avg_intensity_joined['characteristics[organism part]'] == tissue]
# Filter ratio greater than 2
top_marker_skin = top_marker_skin[top_marker_skin['Tissue_Ratio'] > 3]

# Sort by Tissue_Ratio
top_marker_skin = top_marker_skin['Genes'].sort_values(ascending=True).tolist()
top_marker_skin


# Look at HPA


In the search box, you can search multiple genes with this simple syntax:
```
gene_name:ACE2, INS
```

## Q7. 
**From HPA, do you see evidences of the tissue location?** 
Ans. 

## Q8. 
**Similarly, what can be the marker for your cell line?**


In [ ]:
# Visualize protein expression for your markers

# Filter with your avg_intensity_joined_skin['Genes'] from dr_plot_join
dr_plot_join_skin = dr_plot_join[dr_plot_join['Genes'].isin(top_marker_skin)]
# Reorder by Intensity
dr_plot_join_skin

In [ ]:
order = sdrf['factor value[cell line]'].unique().tolist()

# Facet the bar plot by Genes and color by characteristics[organism part]
g = sns.FacetGrid(dr_plot_join_skin, col="Genes", col_wrap=3, sharey=False, height=4)
g.map(sns.barplot, "factor value[cell line]", "Intensity", "characteristics[organism part]", 
      order=order, ci=None, palette="Set2")
g.set_xticklabels(rotation=90)
g.set_axis_labels("Cell Lines", "Intensity")
# g.set(yscale="log")  # Set y-axis to log scale
g.fig.tight_layout()
# 
plt.show()